<a id="title_ID"></a>

[
<img src="https://photutils.readthedocs.io/en/stable/_static/photutils_banner.svg" width=300>
](http://photutils.readthedocs.io/en/stable/index.html)

# Aperture Photometry with `photutils`
---

##### What is aperture photometry?
The most common method to measure the flux of a celestial source is aperture photometry. This kind of photometry measures the amount of flux within a region of an astronomical image of defined shape and size (an aperture) surrounding a source. The ideal aperture would capture all of the flux emitted by a desired source, and none of the flux emitted by the surrounding sky or nearby sources. Especially when performing photometry on image data that includes a number of sources with varying size and shape, it is important to perform aperture corrections to account for imperfect apertures and better constrain photometric errors.

The `photutils` package provides tools for performing photometry with apertures of various shapes.

##### What does this tutorial include?
This tutorial covers how to perform aperture photometry with `photutils`, including the following methods:
* Creating Apertures
    * Circular Apertures
    * Elliptical Apertures
    * Sky Apertures with WCS
* Performing Aperture Photometry
* Calculating Aperture Corrections with Local Background Subtraction

##### Which data are used in this tutorial?
We will be manipulating Hubble eXtreme Deep Field (XDF) data, which was collected using the Advanced Camera for Surveys (ACS) on Hubble between 2002 and 2012. The image we use here is the result of 1.8 million seconds (500 hours!) of exposure time, and includes some of the faintest and most distant galaxies that have ever been observed. 

*The methods demonstrated here are available in narrative form within the `photutils.aperture` [documentation]( http://photutils.readthedocs.io/en/stable/aperture.html).*

<div class="alert alert-block alert-warning">

**Important:** Before proceeding, please be sure to install or update your [AstroConda](https://astroconda.readthedocs.io) distribution. This notebook may not work properly with older versions of AstroConda.

</div>

---

## Import necessary packages

First, let's import packages that we will use to perform arithmetic functions and visualize data:

In [ ]:
from astropy.io import fits
import astropy.units as u
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
import numpy as np
from photutils.background import Background2D, MeanBackground

# Show plots in the notebook
%matplotlib inline

Let's also define some `matplotlib` parameters, such as title font size and the dpi, to make sure our plots look nice. To make it quick, we'll do this by loading a [style file shared with the other photutils tutorials](../photutils_notebook_style.mplstyle) into `pyplot`. We will use this style file for all the notebook tutorials. (See [here](https://matplotlib.org/users/customizing.html) to learn more about customizing `matplotlib`.)

In [ ]:
plt.style.use('../photutils_notebook_style.mplstyle')

## Retrieve data

As described in the introduction, we will be using Hubble eXtreme Deep Field (XDF) data. Since this file is too large to store on GitHub, we will just use `astropy` to directly download the file from the STScI archive: https://archive.stsci.edu/prepds/xdf/ 

(Generally, the best package for web queries of astronomical data is [Astroquery](https://astroquery.readthedocs.io/en/latest/); however, the dataset we are using is a High Level Science Product (HLSP) and thus is not located within a catalog that could be queried with Astroquery.)

In [ ]:
url = 'https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f435w_v1_sci.fits'
with fits.open(url) as hdulist:
    hdulist.info()
    data = hdulist[0].data
    header = hdulist[0].header

As explained in a [previous notebook](../01_background_estimation/01_background_estimation.ipynb) on background estimation, it is important to **mask** these data, as a large portion of the values are equal to zero. We will mask out the non-data portions of the image array, so all of those pixels that have a value of zero don't interfere with our statistics and analyses of the data. 

In [ ]:
# Define the mask
mask = data == 0

Throughout this notebook, we are going to store our images in Python using a `CCDData` object (see [Astropy documentation](http://docs.astropy.org/en/stable/nddata/index.html#ccddata-class-for-images)), which contains a `numpy` array in addition to metadata such as uncertainty, masks, or units. In this case, our data is in electrons (counts) per second.

In [ ]:
unit = u.electron / u.s
xdf_image = CCDData(data, unit=unit, meta=header, mask=mask)

Let's look at the data:

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Set up the normalization and colormap
norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
cmap = plt.get_cmap('viridis')
cmap.set_over(cmap.colors[-1])
cmap.set_under(cmap.colors[0])
cmap.set_bad('white') # Show masked data as white
xdf_image_clipped = np.clip(xdf_image, 1e-4, None) # clip to plot with logarithmic stretch

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)

# Define the colorbar and fix the labels
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')

*Tip: Double-click on any inline plot to zoom in.*

---
## Creating Apertures

With `photutils`, users can create apertures with the following shapes:
<img src="https://dl.boxcloud.com/api/2.0/internal_files/460791262443/versions/487657258443/representations/png_paged_2048x2048/content/1.png?access_token=1!mK2E1OiuUdXBeQpFm1fs2aEIHI4qAiRuDMxlpoH0Ng8FDqBkDzcixwbHAe87KVv5VwoFt4yZRT6Yefdjr_F41vW1n7ORLBDGPcB_4ULHfTLeu7ADmrKwMuqbz3GOf4he8KsIxOKl9z_SuxTmpxp2uGFScMLDUEiUYeEiEgTYkmhCIoVT1OiEkRAzLyUhHIMIscm83ZN8CMgSEZWjxlr3e5fvbAediNGoKR6nJGmCGgj4Daa_ua3RozSw22PbW9uxe8CL79VpVzSnS3pu_7S2pZdqnJYjk-2RyiJ90paTt3Ow8eWR0BAqSfFkWJ69nD8PlNXFiHZf6M3INmMDk9zn4GgUiDIjh1wFU__9rGxy6iGDtjF6JUuGLjT8OqTrTWvf6A3ZE3UzqF12H9Wm_B7LgTE40yQSuBXQGEX88u452kNWoiUsRES4LuS_Rmrw2lxBJd82RW66cO7OYGfSAOyWBjM6l40oqy29cUgfwj2yVdDp2mIL_4Mo9BZLZFN_f8OdxId6VVBEmlK0oKQCEpy-ka0ZjF406_cn0rBu2ABjxIIkihyQ_3zHqs43xdbTj3qFVQ..&box_client_name=box-content-preview&box_client_version=2.8.1" width=700 alt="Examples of circular, elliptical, and rectangular apertures and annuli.">

Each of these can be defined either in pixel coordinates or in celestial coordinates (using a WCS transformation).

It is also possible for users to create custom aperture shapes.

Any aperture object is created by defining its position and size (and, if applicable, orientation). Let's use the `find_peaks` method that we learned about in a [previous notebook](../02_source_detection/02_source_detection.ipynb) to get the positions of sources in our data:

In [ ]:
from photutils import find_peaks
from photutils.centroids import centroid_2dg

In [ ]:
# Calculate statistics
mean, median, std = sigma_clipped_stats(xdf_image.data, sigma=3.0, maxiters=5, mask=xdf_image.mask)

In [ ]:
sources_findpeaks = find_peaks(xdf_image.data, mask=xdf_image.mask, 
                               threshold=20.*std, box_size=30, 
                               centroid_func=centroid_2dg)     
# Display the table
sources_findpeaks

And let's plot the centroids of each of these sources:

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), norm=norm_image)
ax1.scatter(sources_findpeaks['x_centroid'], sources_findpeaks['y_centroid'], s=10, marker='.', 
            lw=1, alpha=0.7, color='r')#facecolor='None', edgecolor='r')

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('find\_peaks Sources')

So thanks to `find_peaks`, we now we know the positions of all our sources. Next, we need to define apertures for each source. First, as the simplest example, let's try using circular apertures of a fixed size.

### Circular Apertures

In [ ]:
from photutils import CircularAperture

In [ ]:
# define the aperture
position = (sources_findpeaks['x_centroid'], sources_findpeaks['y_centroid'])
radius = 10.
circular_aperture = CircularAperture(position, r=radius)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), norm=norm_image)

# Plot the apertures
circular_aperture.plot(color='red', alpha=0.7)

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Circular Apertures')

# Crop to show an inset of the data
ax1.set_xlim(2000, 3000)
ax1.set_ylim(2000, 1000)

As you can see, these circular apertures don't fit our data very well. After all, this is the Hubble eXtreme Deep Field, so there aren't any nice, round, nearby Milky Way stars in this image! 

Let's use ellipses instead, to better match the morphology of the galactic blobs in our image.

### Elliptical Apertures

In [ ]:
from photutils import (detect_sources, source_properties, \
                       EllipticalAnnulus, EllipticalAperture)

In a [previous notebook](../02_source_detection/02_source_detection.ipynb), we showed how you can use the `photutils.detect_sources` [feature](https://photutils.readthedocs.io/en/stable/api/photutils.detect_sources.html) to generate segmentation maps, which identify and label contiguous (connected) objects within an image. Then, with `source_properties` [feature](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.source_properties.html?highlight=source_properties), you can access descriptive properties for each unique object - not just their centroid positions, but also their pixel areas, eccentricities, orientations with respect to the coordinate frame of the image, and more.

Here we'll use the centroid, semimajor axis, semiminor axis, and orientation values from `source_properties` to generate elliptical apertures for each of the sources in our image.

In [ ]:
# Define threshold and minimum object size
threshold = 5. * std
npixels = 15

# Create a segmentation image
segm = detect_sources(xdf_image.data, threshold, npixels)

# Create a catalog using source properties
catalog = source_properties(xdf_image.data, segm)
table = catalog.to_table()

# Display the table
table

In [ ]:
r = 3.  # approximate isophotal extent of semimajor axis

# Create the apertures
elliptical_apertures = []
for obj in catalog:
    position = (obj.xcentroid.value, obj.ycentroid.value)
    a = obj.semimajor_axis_sigma.value * r
    b = obj.semiminor_axis_sigma.value * r
    theta = obj.orientation.value
    elliptical_apertures.append(EllipticalAperture(position, a, b, theta=theta))

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), norm=norm_image)

# Plot the apertures
for aperture in elliptical_apertures:
    aperture.plot(color='red', alpha=0.7, ax=ax1)

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Elliptical Apertures')

# Crop to show an inset of the data
ax1.set_xlim(2000, 3000)
ax1.set_ylim(2000, 1000)

Clearly, these custom-made elliptical apertures fit our XDF galaxies much better than the one-size-fits-all circular apertures from before.

### Sky Coordinates & Apertures

At the moment, the positions of our apertures are in pixels, relative to our data array. However, if you need aperture positions in terms of celestial coordinates, `photutils` also includes aperture objects that can be integrated with Astropy's `SkyCoords`.

Fortunately this is extremely easy when we use the [World Coordinate System (WCS)](http://docs.astropy.org/en/stable/wcs/) to produce a WCS object from the header of the FITS file containing our image, and then the `to_sky()` method to transform our `EllipticalAperture` objects into `SkyEllipticalAperture` objects.

In [ ]:
from astropy.wcs import WCS

In [ ]:
wcs = WCS(header)
sky_elliptical_apertures = [ap.to_sky(wcs) for ap in elliptical_apertures]

If we wanted to generate `SkyEllipticalAperture` objects from the get-go, we could have used that WCS object in the following way:

In [ ]:
from photutils import SkyEllipticalAperture
from astropy.coordinates import SkyCoord

r = 3.  # approximate isophotal extent of semimajor axis

# Create the apertures
sky_elliptical_apertures = []
for obj in catalog:
    # Convert the centroids into RA/Dec using WCS
    ra, dec = wcs.all_pix2world(obj.xcentroid.value, obj.ycentroid.value, 0)
    # Convert the positions to an Astropy SkyCoord object, with units!
    sky_position = SkyCoord(ra, dec, unit=u.deg)
    
    # Define the elliptical parameters, now with units
    a = obj.semimajor_axis_sigma.value * r * u.pix
    b = obj.semiminor_axis_sigma.value * r * u.pix
    theta = obj.orientation.value  * u.rad
    
    # Convert the theta from radians from X axis to the radians from North 
    x_to_north_angle = (90. + header['ORIENTAT']) * u.deg
    x_to_north_angle_rad = x_to_north_angle.to_value(u.rad) * u.rad
    theta -= x_to_north_angle_rad
    
    # Define the apertures
    ap = SkyEllipticalAperture(sky_position, a, b, theta=theta)
    sky_elliptical_apertures.append(ap)

Unfortunately, you can't plot SkyApertures. However, you can use them just as easily to perform aperture photometry!

----
## Performing Aperture Photometry

Now that we have aperture objects that fit our data reasonably well, we can finally perform photometry with the `aperture_photometry` function. This function takes the following arguments:

* **`data`** - the background-subtracted data array on which to perform photometry.
* **`apertures`** - an aperture object containing the aperture(s) to use for the photometry.
* **`error`** (optional) - an array of values that represent the pixel-wise Gaussian 1-sigma errors of the input data.
* **`mask`** (optional) - a mask for the `data` to exclude certain pixels from calculations.
* **`method`** (optional) - how to place the aperture(s) onto the pixel grid (see below).
* **`unit`** (optional) - unit of `data` and `error`.
* **`wcs`** (optional) - the WCS transformation to use if `apertures` is a `SkyAperture` object. 
* **`subpixels`** (optional) - the factor by which pixels are resampled (see below).

The following methods are the options for how to place apertures onto the data pixel grid:

* **exact** (default) - calculate the exact fractional overlap of each aperture for each overlapping pixel. This method is the most accurate, but will also take the longest. 
* **center** - a pixel is either entirely in or entirely out of the aperture, depending on whether the pixel center is inside or outside of the aperture.
* **subpixel** - a pixel is divided into `subpixels` x `subpixels` subpixels, each of which are considered to be entirely in or out of the aperture depending on whether its center is in or out of the aperture. 

In [ ]:
from photutils import aperture_photometry
from astropy.table import QTable

Let's see what this looks like using the first aperture in our image:

In [ ]:
# The CCDData mask will be automatically applied
phot_datum = aperture_photometry(xdf_image, elliptical_apertures[0]) 
phot_datum

The `aperture_sum` value is what reports the number of electron counts within the aperture: 3.47 e<sup>–</sup>/s.

And, just as a check, to make sure our sky apertures give basically the same answer...

In [ ]:
# The CCDData mask will be automatically applied
sky_phot_datum = aperture_photometry(xdf_image, sky_elliptical_apertures[0], wcs=wcs)
sky_phot_datum

Woohoo!

Unfortunately for our purposes, the `aperture_photometry` function can be only used alone for one of the two cases:
* Identical apertures at distinct positions (e.g. circular apertures with `r = 3` for many sources)
* Distinct apertures at identical positions (e.g. two circular apertures with `r = 3` and `r = 5` for one source)

Since our elliptical apertures are distinct apertures at distinct positions, we need to do a little more work to get a single table of photometric values.

(This step will take a while, almost 5 minutes, so hang tight!)

In [ ]:
# The CCDData mask will be automatically applied
phot_table = aperture_photometry(xdf_image, elliptical_apertures[0])
id = 1
for aperture in elliptical_apertures[1:]:
    id += 1
    phot_row = aperture_photometry(xdf_image, aperture)[0]
    phot_row[0] = id
    phot_table.add_row(phot_row)

Let's take a look at all these apertures we've made:

In [ ]:
# Display the table
phot_table

There's only so much you can learn from looking at a table of numbers, so let's explore alternate ways to examine these data.

In [ ]:
plt.figure(figsize=(8, 5))

values = [phot.value for phot in phot_table['aperture_sum']]
logbins=bins = 10.**(np.linspace(-1, 2, 100))
plt.hist(values, bins=logbins)

plt.yscale('log')
plt.xscale('log')
plt.title('Histogram of Source Photometry')
plt.xlabel(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')))
plt.ylabel('Number of Sources')

In [ ]:
plt.figure(figsize=(8, 5))

plt.scatter(table['area'], values, alpha=0.5)

plt.yscale('log')
plt.xscale('log')
plt.title('Count Rate v. Aperture Area')
plt.xlabel('Aperture Area [pixels$^2$]')
plt.xlabel(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')))

<div class="alert alert-block alert-info">

<h3>Exercise:</h3><br>

Re-calculate the photometry for these elliptical apertures - or just a subset of them - using the `subpixel` aperture placement method instead of the default `exact` method. How does this affect the count sum calculated for those apertures?

</div>

---
# Aperture Corrections

We've done photometry with some lovely apertures, but unfortunately even using elliptical apertures with unique sizes and orientations does not account for an important source of extraneous flux: the sky background.

## Local Background Subtraction

In the [background estimation notebook](../01_background_estimation/01_background_estimation.ipynb), we explored how to perform global background subtraction of image data with `photutils`. However, you can also use `photutils` to perform local background estimations for aperture corrections.

To estimate the local background for each aperture, measure the counts within annulus apertures around (but not including!) each source. In our example, we defined elliptical apertures with `r = 3` to measure the counts within each source. To calculate the background for each source, let's measure the counts elliptical annuli between `r = 3.5` and `r = 5`.

In [ ]:
r_in = 3.5  # approximate isophotal extent of inner semimajor axis
r_out = 5.  # approximate isophotal extent of inner semimajor axis

# Create the apertures
elliptical_annuli = []
for obj in catalog:
    position = (obj.xcentroid.value, obj.ycentroid.value)
    a_in = obj.semimajor_axis_sigma.value * r_in
    a_out = obj.semimajor_axis_sigma.value * r_out
    b_out = obj.semiminor_axis_sigma.value * r_out
    theta = obj.orientation.value
    elliptical_annuli.append(EllipticalAnnulus(position, a_in, a_out, b_out, theta=theta))

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image)

# Plot the apertures
for aperture in elliptical_annuli:
    aperture.plot(color='red', alpha=0.4, ax=ax1, fill=True)
for aperture in elliptical_apertures:
    aperture.plot(color='white', alpha=0.7, ax=ax1)

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Elliptical Annuli')

# Crop to show an inset of the data
ax1.set_xlim(2000, 3000)
ax1.set_ylim(2000, 1000)

## Calculating Aperture Corrections

Now that our apertures have been defined, we can do photometry with them to estimate and account for the background. The aperture correction is calculated by:
- Calculating the count rate within each annulus using `aperture_photometry`
- Dividing each annulus' count rate by each annulus' area to get the mean background value for each annulus
- Taking the mean of those annulus means to get a mean background value for the entire image
- Multiplying the global background mean value times the area of each elliptical photometric aperture, to get the estimated background count rate within each aperture
- Subtracting the estimated background count rate from the photometric count rate for each aperture

(Just like when we did photometry with the elliptical apertures above, the below step will take almost 5 minutes.)

In [ ]:
# The CCDData mask will be automatically applied
bkg_phot_table = aperture_photometry(xdf_image, elliptical_annuli[0])
id = 1
for aperture in elliptical_annuli[1:]:
    id += 1
    phot_row = aperture_photometry(xdf_image, aperture)[0]
    phot_row[0] = id
    bkg_phot_table.add_row(phot_row)

In [ ]:
# Display table
bkg_phot_table

In [ ]:
# Calculate the mean background level (per pixel) in the annuli 
bkg_area = [annulus.area() for annulus in elliptical_annuli]
bkg_mean_per_aperture = bkg_phot_table['aperture_sum'].value / bkg_area
bkg_mean = np.average(bkg_mean_per_aperture) * (u.electron / u.s)
print('Background mean:', bkg_mean)

# Calculate the total background within each elliptical aperture
bkg_sum = bkg_mean * table['area'].value

# Subtract the background from the original photometry
flux_bkgsub = phot_table['aperture_sum'] - bkg_sum

# Add this as a column to the original photometry table
phot_table['aperture_sum_bkgsub'] = flux_bkgsub

You might have noticed that these background count rates are *really* small. In this case, this is to be expected - since our example XDF data is a high-level science product (HLSP) that already has already been background-subtracted.

Finally, let's see the difference between our original count rates and our background-subtracted count rates (it should be small for us!):

In [ ]:
# Display table
phot_table

In [ ]:
plt.figure(figsize=(8, 5))

values = [phot.value for phot in phot_table['aperture_sum']]
values_bkgsub = [phot.value for phot in phot_table['aperture_sum_bkgsub']]
logbins=bins = 10.**(np.linspace(-1, 2, 100))
plt.hist(values, bins=logbins, alpha=0.7, label='Original photometry')
plt.hist(values_bkgsub, bins=logbins, alpha=0.7, label='Background-subtracted')

plt.yscale('log')
plt.xscale('log')
plt.title('Histogram of Source Photometry')
plt.xlabel(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')))
plt.legend()

---
# Conclusions

The `photutils` package provides a comprehensive toolkit for astronomers to perform aperture photometry, including customizable aperture shapes that allow for more precise photometry and easy photometric correction.

**To continue with this `photutils` tutorial, go on to the [PSF photometry notebook](../04_psf_photometry/04_psf_photometry.ipynb).**

---
## Additional Resources
For more examples and details, please visit the [photutils](http://photutils.readthedocs.io/en/stable/index.html) documentation.

---
## About this Notebook
**Authors:** Lauren Chambers (lchambers@stsci.edu), Erik Tollerud (etollerud@stsci.edu), Tom Wilson (towilson@stsci.edu) Clare Shanahan (cshanahan@stsci.edu)
<br>**Updated:** May 2019

[Top of Page](#title_ID)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>